# Benchmarking and Ablation Studies

Research-style notebook for systematic method comparison and ablation experiments.

**Estimated Runtime:** 15-30 minutes

In [ ]:
# !pip install promethium-seismic==1.0.1

In [ ]:
import promethium
from promethium import (
    evaluate_reconstruction,
    generate_synthetic_traces,
    add_noise,
    bandpass_filter,
    set_seed,
)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

set_seed(42)
print(f"Promethium {promethium.__version__}")

## 1. Define Methods to Compare

In [ ]:
def method_raw(data):
    """No processing (baseline)."""
    return data

def method_gaussian(data, sigma=2):
    """Gaussian smoothing."""
    return np.array([gaussian_filter1d(t, sigma=sigma) for t in data])

def method_bandpass(data, fs=250):
    """Bandpass filtering."""
    return np.array([bandpass_filter(t, 5, 60, fs) for t in data])

methods = {
    'raw': method_raw,
    'gaussian_s1': lambda d: method_gaussian(d, sigma=1),
    'gaussian_s2': lambda d: method_gaussian(d, sigma=2),
    'gaussian_s4': lambda d: method_gaussian(d, sigma=4),
    'bandpass': method_bandpass,
}

## 2. Run Benchmark

In [ ]:
# Generate test data
clean_data, meta = generate_synthetic_traces(n_traces=50, n_samples=500, seed=42)

# Test with different noise levels
noise_levels = [0.1, 0.2, 0.3, 0.5]
results = []

for noise in noise_levels:
    noisy = add_noise(clean_data, noise_level=noise, seed=42)
    
    for method_name, method_fn in methods.items():
        recovered = method_fn(noisy)
        metrics = evaluate_reconstruction(clean_data, recovered)
        metrics['method'] = method_name
        metrics['noise_level'] = noise
        results.append(metrics)

df = pd.DataFrame(results)
print(f"Collected {len(results)} results")

## 3. Results Summary

In [ ]:
# Pivot table for SNR
pivot = df.pivot_table(values='snr', index='method', columns='noise_level')
print("SNR (dB) by Method and Noise Level:")
print(pivot.round(2).to_string())

In [ ]:
# Plot results
fig, ax = plt.subplots(figsize=(10, 6))

for method in methods.keys():
    method_data = df[df['method'] == method]
    ax.plot(method_data['noise_level'], method_data['snr'], 'o-', label=method)

ax.set_xlabel('Noise Level')
ax.set_ylabel('SNR (dB)')
ax.set_title('Method Performance vs Noise Level')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Ablation: Gaussian Sigma Effect

In [ ]:
# Test different sigma values
noisy = add_noise(clean_data, noise_level=0.3, seed=42)
sigmas = [0.5, 1, 2, 3, 4, 5, 6, 8, 10]

ablation = []
for sigma in sigmas:
    recovered = method_gaussian(noisy, sigma=sigma)
    metrics = evaluate_reconstruction(clean_data, recovered)
    metrics['sigma'] = sigma
    ablation.append(metrics)

abl_df = pd.DataFrame(ablation)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(abl_df['sigma'], abl_df['snr'], 'bo-')
axes[0].set_xlabel('Sigma')
axes[0].set_ylabel('SNR (dB)')
axes[0].set_title('SNR vs Gaussian Sigma')
axes[0].grid(True, alpha=0.3)

axes[1].plot(abl_df['sigma'], abl_df['ssim'], 'go-')
axes[1].set_xlabel('Sigma')
axes[1].set_ylabel('SSIM')
axes[1].set_title('SSIM vs Gaussian Sigma')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

best_idx = abl_df['snr'].idxmax()
print(f"Optimal sigma: {abl_df.loc[best_idx, 'sigma']} (SNR: {abl_df.loc[best_idx, 'snr']:.2f} dB)")

## Summary

This notebook demonstrated:
1. Systematic method comparison
2. Multi-condition benchmarking
3. Parameter ablation studies
4. Result visualization and analysis